#  Simple tool to analyze data from https://data.europa.eu/

The EU Open Data Portal (EU ODP) aims to encourage the use of EU datasets for building third-party applications.

**Note:** This is a Jupyter notebook which is also available as its executable export as a Python 3 script (therefore with automatically generated comments).

# Libraries

In [1]:
# Sys import
import sys, os, re
# Common imports
import math
import numpy             as NP
import numpy.random      as RAND
import scipy.stats       as STATS
from scipy import sparse
from scipy import linalg

# Better formatting functions
from IPython.display import display, HTML
from IPython import get_ipython

import matplotlib        as MPL
import matplotlib.pyplot as PLT
import seaborn as SNS
SNS.set(font_scale=1)

# Python programming
from itertools import cycle
from time import time
import datetime

# Using pandas
import pandas as PAN
import xlrd

In [2]:
import warnings
warnings.filterwarnings('ignore')
print("For now, reduce python warnings, I will look into this later")

For now, reduce python warnings, I will look into this later


### Import my own modules
The next cell attempts to give user some information if things improperly setup.
Intended to work both in Jupyter and when executing the Python file directly.

In [3]:
if not get_ipython() is None and os.path.abspath("../source/") not in sys.path:
    sys.path.append(os.path.abspath("../source/"))
try:
    from lib.utilities     import *
    from lib.figureHelpers import *
    from lib.DataMgrRdf        import *
    import lib.basicDataCTE as DCTE
except Exception as err:
    print("Could not find library 'lib' with contents 'DataGouvFr' ")
    if get_ipython() is None:
        print("Check the PYTHONPATH environment variable which should point to 'source' wich contains 'lib'")
    else:
        print("You are supposed to be running in JupySessions, and '../source/lib' should exist")
    raise err

Loaded lib
For now, reduce python warnings, I will look into this later


## Check environment

It is expected that:
- your working directory is named `JupySessions`, 
- that it has subdirectories 
   - `images/*` where generated images may be stored to avoid overcrowding. 
- At the same level as your working dir there should be directories 
   - `../data` for storing input data and 
   - `../source` for python scripts.
   
My package library is in `../source/lib`, and users running under Python (not in Jupyter) should
set their PYTHONPATH to include "../source" ( *or whatever appropriate* ).

In [4]:
checkSetup(chap="Chap03")
ImgMgr = ImageMgr(chapdir="Chap03")

# Load Data

## Functions

## Load CSV and XLSX data from remote 
The `dataFileVMgr` will manage a cache of data files in `../dataEURdf`, the data will be downloaded
from  https://data.europa.eu using the SPARQL query endpoint. The meta information is stored/cached  in `../dataEURdf/.cache*`

We check what is in the cache/data directory; for each file, we identify the latest version, 
and list this below to make sure. The file name will usually contain a time stamp.

<FONT COLOR="RED">TO BE CHECKED For the files used in this notebook, the latest version is used/loaded irrespective of the
timestamp used in the notebook.</FONT>

In [5]:
dataFileVMgr = manageAndCacheDataFilesRdfEU( "../dataEURdf")
dataFileVMgr.getRemoteInfo()

Loaded pickle from ../dataEURdf/.cache.rdf, loaded 00h 26m 37s ago (336032 elts)
cache metadata:{   'HTTP:Hdr:resp': {'Date': 'Thu, 07 May 2020 13:41:19 GMT', 'Content-Type': 'application/rdf+xml; charset=UTF-8', 'Access-Control-Allow-Origin': '*', 'Server': 'Europa', 'Connection': 'close', 'Content-Encoding': 'gzip'},
    'HTTP:Hdr:rqt': {'User-Agent': 'python-requests/2.23.0', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/rdf+xml', 'Connection': 'keep-alive', 'Content-Type': 'application/rdf+xml', 'Content-Length': '0'},
    'HTTP:Query:Sparql': '\n'
                         'PREFIX http: <http://www.w3.org/2011/http#>\n'
                         'PREFIX dcat: <http://www.w3.org/ns/dcat#>\n'
                         'PREFIX odp:  <http://data.europa.eu/euodp/ontologies/ec-odp#>\n'
                         'PREFIX dc: <http://purl.org/dc/terms/>\n'
                         'PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>\n'
                         'PREFIX foaf: <http://x

Loading ../dataEURdf/.cache.prdf as xml... 

Parsed ../dataEURdf/.cache.prdf


done.	(17586 triples)


This can be long, the SPARQL processor used is not fast

In [6]:
dataFileVMgr.updatePrepare()
dataFileVMgr.cacheUpdate()

## Get some understanding of the available resource

## Dig into the data

In [7]:
print("Most recent versions of files in data directory:")
for f in dataFileVMgr.listMostRecent(nonTS=True) :
    print(f"\t{f}")

Most recent versions of files in data directory:
	.cache.prdf
	.cache.qhash
	.cache.rdf
	covid-19-coronavirus-data.csv


In [8]:
dataFileVMgr.nonTSFiles

{'.cache.rdf': None,
 'covid-19-coronavirus-data.csv': None,
 '.cache.prdf': None,
 '.cache.qhash': None}

In [9]:
last = lambda x: dataFileVMgr.getRecentVersion(x,default=True)